# Audio Hybrid Neuroevolution Notebook

This notebook implements a hybrid neuroevolution process for audio classification (Parkinson detection). The system combines genetic algorithms with 1D convolutional neural networks to evolve optimal architectures for audio processing.

## Main Features:
- **Hybrid genetic algorithm**: Combines architecture and weight evolution
- **1D Convolutional Networks**: Optimized for audio waveform processing
- **Parallel 5-Fold Cross-Validation**: Each individual is evaluated on all 5 folds IN PARALLEL (fitness = average accuracy)
- **Multi-threading**: Folds are trained simultaneously in separate threads for faster evaluation
- **Adaptive mutation**: Dynamic mutation rate based on population diversity
- **Audio dataset support**: Loads .npy files with train/val/test splits
- **Intelligent stopping criteria**: By target fitness or maximum generations
- **Complete visualization**: Shows progress and final best architecture

## Objectives:
1. Create initial population of 1D CNN architectures
2. Evaluate fitness of each individual using **parallel 5-fold CV** (robust and faster with threading)
3. Select best architectures (elitism)
4. Apply crossover and mutation to create new generation
5. Repeat process until convergence
6. Display the best architecture found for Parkinson classification

**✅ Performance**: Multi-threaded 5-fold CV provides robustness against overfitting while being much faster than sequential training.


---
## ✨ CONFIGURACIÓN ACTUAL DEL DATASET ✨

**Dataset configurado**: `files_all_real_syn_n` (Datos Reales + Sintéticos Mezclados)

Este notebook está configurado para usar el **nuevo dataset** que combina:
- 🎵 **Datos Reales**: Audios originales de pacientes  
- 🤖 **Datos Sintéticos**: Audios generados por GANs (BigVSAN 40_1e5)

**Ventajas de este dataset**:
- Mayor diversidad de datos para entrenamiento
- Combina la autenticidad de datos reales con la variedad de datos generados
- Ideal para mejorar la generalización del modelo
- Estratificación balanceada entre clases (control/patológico)

**🚀 Parallel 5-Fold Cross-Validation durante la Evolución**: 
- **CADA** individuo se evalúa en **TODOS** los 5 folds **EN PARALELO**
- Los 5 folds se entrenan **simultáneamente** en threads separados
- El fitness es el **promedio** de accuracy de los 5 folds
- ✅ **Mucho más rápido** que entrenamiento secuencial
- ✅ **Más robusto** - evita sobreajuste a un fold específico

**📊 Evaluación Final**: 
- Al terminar la evolución, la mejor arquitectura se vuelve a evaluar con 5-fold CV
- Se reportan métricas completas (accuracy, sensitivity, specificity, F1, AUC)

Para cambiar el dataset, modifica los parámetros `dataset_id` y `fold_id` en la celda de **Configuración** (Sección 2).

---

## 1. Required Libraries Import

In [ ]:
# Install all necessary libraries
import subprocess
import sys

def install_package(package):
    """Installs a package using pip if not available."""
    try:
        __import__(package.split('==')[0].split('[')[0])
        print(f"OK {package.split('==')[0]} is already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"OK {package} installed correctly")

# List of required packages
required_packages = [
    "torch>=2.0.0",
    "torchvision>=0.15.0",
    "numpy>=1.21.0",
    "matplotlib>=3.5.0",
    "seaborn>=0.11.0",
    "tqdm>=4.64.0",
    "jupyter>=1.0.0",
    "ipywidgets>=8.0.0"
]

print("Starting dependency installation for Hybrid Neuroevolution...")
print("=" * 60)

for package in required_packages:
    install_package(package)

print("\nAll dependencies have been verified/installed")
print("Restart the kernel if this is the first time installing torch")
print("=" * 60)

# Verify PyTorch installation
try:
    import torch
    print(f"\nPyTorch {torch.__version__} installed correctly")
    print(f"CUDA available: {'Yes' if torch.cuda.is_available() else 'No'}")
    if torch.cuda.is_available():
        print(f"GPU detected: {torch.cuda.get_device_name(0)}")
        print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory // 1024**3} GB")
except ImportError:
    print("ERROR: PyTorch could not be installed correctly")
    print("Try installing manually with: pip install torch torchvision")

In [ ]:
# Main imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Scientific libraries
import numpy as np
import random
import copy
import json
import os
from typing import Dict, List, Tuple, Any
from datetime import datetime
import uuid

# Threading for parallel fold training
import threading
from queue import Queue
from concurrent.futures import ThreadPoolExecutor, as_completed

# Visualization and progress
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Configure logging
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configure seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

# Configure device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device configured: {device}")
print(f"PyTorch version: {torch.__version__}")

# Suppress unnecessary warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

## 2. System Configuration and Parameters

In [ ]:
# Main genetic algorithm configuration (updated for adaptive mutation & moderate elitism)
CONFIG = {
    # Genetic algorithm parameters
    'population_size': 50,            # Population size
    'max_generations': 1000,            # Maximum number of generations
    'fitness_threshold': 95.0,        # Target fitness (% accuracy) - Adjusted for audio

    # Adaptive mutation parameters
    'base_mutation_rate': 0.35,       # Starting mutation rate (moderate)
    'mutation_rate_min': 0.10,        # Lower bound for adaptive mutation
    'mutation_rate_max': 0.80,        # Upper bound for adaptive mutation
    'current_mutation_rate': 0.35,    # Will be updated dynamically each generation

    'crossover_rate': 0.99,           # Crossover rate
    'elite_percentage': 0.2,          # Moderate elitism (20%) instead of 40%

    # Dataset selection (AUDIO ONLY)
    'dataset': 'AUDIO',               # Audio dataset for Parkinson classification

    # Dataset parameters for audio
    'num_channels': 1,                # Input channels (1 for audio waveform)
    'sequence_length': 240000,        # Audio sequence length (will be auto-detected)
    'num_classes': 2,                 # Number of classes (control vs pathological)
    'batch_size': 32,                 # Batch size for audio
    'test_split': 0.2,                # Validation percentage

    # Training parameters
    'num_epochs': 30,                 # Max training epochs per evaluation (may stop earlier)
    'learning_rate': 0.001,           # Base learning rate
    'early_stopping_patience': 200,   # Max batches per epoch (quick partial epoch)

    # Epoch-level early stopping
    'epoch_patience': 3,              # Stop if no significant improvement after N evaluations
    'improvement_threshold': 0.5,     # Minimum (absolute) accuracy gain (%) to reset patience

    # Generation-level early stopping 
    'early_stopping_generations': 10, # Stop if no improvement in X generations
    'min_improvement_threshold': 0.1, # Minimum fitness improvement (%) to reset counter

    # Allowed architecture range for 1D Conv
    'min_conv_layers': 1,
    'max_conv_layers': 5,             # Less layers for 1D audio
    'min_fc_layers': 1,
    'max_fc_layers': 3,               # Less FC layers
    'min_filters': 8,
    'max_filters': 128,               # Adjusted for 1D
    'min_fc_nodes': 64,
    'max_fc_nodes': 512,              # Smaller for audio classification

    # Audio dataset configuration (OS-independent paths)
    
    'dataset_id': 'all_real_syn_n',   # Dataset ID - Mixed real + synthetic data
    'fold_id': 'all_real_syn_n',      # Fold ID for files
    'num_folds': 5,                   # Number of folds (all used during evolution)
    'data_path': os.path.join('data', 'sets', 'folds_5'),  # OS-independent path
    'normalization': {'mean': (0.0,), 'std': (1.0,)}  # Audio normalization
}

# Activation function mapping
ACTIVATION_FUNCTIONS = {
    'relu': nn.ReLU,
    'leaky_relu': nn.LeakyReLU,
    'tanh': nn.Tanh,
    'sigmoid': nn.Sigmoid,
    'selu': nn.SELU,
}

# Optimizer mapping
OPTIMIZERS = {
    'adam': optim.Adam,
    'adamw': optim.AdamW,
    'sgd': optim.SGD,
    'rmsprop': optim.RMSprop,
}

print("Configuration loaded (adaptive mutation enabled, 1D Conv for audio):")
print(f"   Dataset: Audio (Parkinson Classification)")
for key, value in CONFIG.items():
    if key not in ['normalization']:  # Hide normalization details
        print(f"   {key}: {value}")
print(f"\nAvailable activation functions: {list(ACTIVATION_FUNCTIONS.keys())}")
print(f"Available optimizers: {list(OPTIMIZERS.keys())}")
print(f"\nOS-independent path configured: {CONFIG['data_path']}")


### Información sobre el Dataset de Audio

**Dataset de Audio para Clasificación de Parkinson**: 
- Archivos de audio de voz (clasificación Parkinson)
- Datos 1D de forma de onda procesada
- Estructura: archivos .npy con train/val/test splits
- Dificultad: **Alta** - Clasificación médica
- Fitness objetivo recomendado: >85%
- Clases: Control vs Pathological
- Formato de archivos: `{data_path}/files_{fold_id}/X_train_{dataset_id}_fold_{fold}.npy`
- Arquitectura: Conv1D -> BatchNorm1D -> Activation -> MaxPool1D -> FC Layers

**Configuración del Dataset:**
- Modifica los parámetros en la celda de configuración:
  - `dataset_id`: ID del dataset (ej: 'all_real_syn_n')
  - `fold_id`: ID de la carpeta de folds (ej: 'all_real_syn_n')
  - `data_path`: Ruta base a los datos (usa `os.path.join` para compatibilidad multiplataforma)

**🔄 Uso de 5-Fold CV:**
- **Durante la evolución**: Cada individuo se evalúa en los 5 folds automáticamente
- **No se necesita** especificar `current_fold` (se usan todos)
- El fitness es el **promedio** de los 5 folds

**Nota sobre Rutas:**
- Las rutas son **independientes del sistema operativo** (Windows/Linux/Mac)
- Usa `os.path.join()` para construir rutas compatibles
- Ejemplo: `os.path.join('data', 'sets', 'folds_5')` funciona en cualquier OS

---

### Tipos de Carpetas de Folds Disponibles (generadas por `create_5_folds.ipynb`)

El notebook `generating_csv/create_5_folds.ipynb` genera **5 tipos de carpetas** con diferentes combinaciones de datos **reales** y **sintéticos** (generados por GANs) para experimentación:

#### 1. **`files_real_N`** - Solo Datos Reales
   - **Train**: Datos reales (`test_together_N`)
   - **Test**: Datos reales (`test_together_N`)
   - **Uso**: Baseline con datos 100% reales
   - **fold_id**: `'real_N'`
   - **dataset_id**: `'real_N'`

#### 2. **`files_real_40_1e5_N`** - Entrenamiento Sintético, Test Real
   - **Train**: Datos sintéticos (`generated_together_train_40_1e5_N`)
   - **Test**: Datos reales (`test_together_N`)
   - **Uso**: Evaluar si modelos entrenados con sintéticos generalizan a datos reales
   - **fold_id**: `'40_1e5_N'`
   - **dataset_id**: `'40_1e5_N'`

#### 3. **`files_syn_40_1e5_N`** - Solo Datos Sintéticos (mismo conjunto)
   - **Train**: Datos sintéticos (`generated_together_train_40_1e5_N`)
   - **Test**: Datos sintéticos (`generated_together_train_40_1e5_N`)
   - **Uso**: Evaluar capacidad del modelo con datos 100% sintéticos
   - **fold_id**: `'40_1e5_N'`
   - **dataset_id**: `'40_1e5_N'`

#### 4. **`files_syn_1_N`** - Entrenamiento Sintético, Test Sintético Diferente
   - **Train**: Datos sintéticos (`generated_together_train_40_1e5_N`)
   - **Test**: Datos sintéticos diferentes (`test_together_syn_1_N`)
   - **Uso**: Evaluar generalización entre diferentes conjuntos sintéticos
   - **fold_id**: `'40_1e5_N'`
   - **dataset_id**: `'40_1e5_N'`

#### 5. **`files_syn_all_N`** - Solo Datos Reales (mal nombrado probablemente)
   - **Train**: Datos reales (`test_together_N`)
   - **Test**: Datos reales (`test_together_N`)
   - **Uso**: Similar a `files_real_N` (posible duplicado o error de nomenclatura)
   - **fold_id**: `'40_1e5_N'`
   - **dataset_id**: `'40_1e5_N'`

#### 6. **`files_all_real_syn_n`** - ✨ Datos Reales + Sintéticos Mezclados ✨ **(NUEVO)**
   - **Train**: Datos reales + sintéticos mezclados
   - **Validation**: Datos reales + sintéticos mezclados
   - **Test**: Datos reales + sintéticos mezclados
   - **Uso**: Entrenar y evaluar con una mezcla equilibrada de datos reales y generados por GANs
   - **fold_id**: `'all_real_syn_n'`
   - **dataset_id**: `'all_real_syn_n'`
   - **Ventajas**: Combina diversidad de datos sintéticos con autenticidad de datos reales
   - **Configuración actual**: 🔵 **ESTE ES EL DATASET CONFIGURADO POR DEFECTO**

**Nota**: Cada carpeta contiene 5 folds de validación cruzada con:
- `X_train_{dataset_id}_fold_{1-5}.npy` y `y_train_{dataset_id}_fold_{1-5}.npy`
- `X_val_{dataset_id}_fold_{1-5}.npy` y `y_val_{dataset_id}_fold_{1-5}.npy`
- `X_test_{dataset_id}_fold_{1-5}.npy` y `y_test_{dataset_id}_fold_{1-5}.npy`

## 3. Dataset Loading and Preprocessing

In [ ]:
def load_dataset(config: dict) -> Tuple[DataLoader, DataLoader]:
    """
    Loads the audio dataset according to configuration.
    Returns train_loader and test_loader.
    """
    
    print(f"Loading audio dataset from: {config['data_path']}")
    print(f"Dataset ID: {config['dataset_id']}, Fold: {config['current_fold']}")
    
    # Construct paths following ResNet convention
    fold_files_directory = os.path.join(
        config['data_path'], 
        f"files_{config['fold_id']}"
    )
    
    # Check if directory exists
    print(f"\nChecking data directory...")
    print(f"   Looking for: {os.path.abspath(fold_files_directory)}")
    
    if not os.path.exists(fold_files_directory):
        print(f"\n❌ ERROR: Directory not found!")
        print(f"   Expected: {os.path.abspath(fold_files_directory)}")
        
        # Try to find the correct path
        possible_paths = [
            os.path.join('..', 'data', 'sets', 'folds_5', f"files_{config['fold_id']}"),
            os.path.join('data', 'sets', 'folds_5', f"files_{config['fold_id']}"),
            os.path.join('.', 'data', 'sets', 'folds_5', f"files_{config['fold_id']}"),
        ]
        
        print(f"\nSearching for data in alternative locations:")
        for path in possible_paths:
            abs_path = os.path.abspath(path)
            exists = os.path.exists(path)
            print(f"   {'✓' if exists else '✗'} {abs_path}")
            if exists:
                fold_files_directory = path
                print(f"\n✓ Found data at: {os.path.abspath(fold_files_directory)}")
                break
        else:
            raise FileNotFoundError(
                f"\n❌ Could not find data directory!\n"
                f"   Tried paths:\n" + 
                "\n".join([f"      - {os.path.abspath(p)}" for p in possible_paths]) +
                f"\n\n   Please check:\n"
                f"      1. CONFIG['data_path'] is correct\n"
                f"      2. The data files exist\n"
                f"      3. The fold_id '{config['fold_id']}' is correct\n"
            )
    else:
        print(f"   ✓ Directory found: {os.path.abspath(fold_files_directory)}")
    
def load_dataset(config: dict):
    """
    Verifica que los datos existen y carga el primer fold para detectar sequence_length.
    Durante la evolución, cada individuo cargará todos los folds automáticamente.
    
    Args:
        config: Diccionario de configuración
    
    Returns:
        None (solo actualiza config['sequence_length'])
    """
    print("\n" + "="*60)
    print("VERIFICANDO DISPONIBILIDAD DE DATOS")
    print("="*60)
    print(f"Dataset ID: {config['dataset_id']}, Verificando los 5 folds...")
    
    # Build directory path
    fold_files_directory = os.path.join(
        config['data_path'], 
        f"files_{config['fold_id']}"
    )
    
    print(f"   Looking for: {os.path.abspath(fold_files_directory)}")
    
    # If directory not found, try alternative locations
    if not os.path.exists(fold_files_directory):
        possible_paths = [
            os.path.join('..', 'data', 'sets', 'folds_5', f"files_{config['fold_id']}"),
            os.path.join('data', 'sets', 'folds_5', f"files_{config['fold_id']}"),
            os.path.join('.', 'data', 'sets', 'folds_5', f"files_{config['fold_id']}"),
        ]
        
        print(f"\nSearching for data in alternative locations:")
        for path in possible_paths:
            abs_path = os.path.abspath(path)
            exists = os.path.exists(path)
            print(f"   {'✓' if exists else '✗'} {abs_path}")
            if exists:
                fold_files_directory = path
                print(f"\n✓ Found data at: {os.path.abspath(fold_files_directory)}")
                break
        else:
            raise FileNotFoundError(
                f"\n❌ Could not find data directory!\n"
                f"   Tried paths:\n" + 
                "\n".join([f"      - {os.path.abspath(p)}" for p in possible_paths]) +
                f"\n\n   Please check:\n"
                f"      1. CONFIG['data_path'] is correct\n"
                f"      2. The data files exist\n"
                f"      3. The fold_id '{config['fold_id']}' is correct\n"
            )
    else:
        print(f"   ✓ Directory found: {os.path.abspath(fold_files_directory)}")
    
    dataset_id = config['dataset_id']
    
    # Check that all 5 folds exist
    print(f"\nChecking for all 5 folds...")
    all_folds_ok = True
    
    for fold_num in range(1, 6):
        required_files = [
            f'X_train_{dataset_id}_fold_{fold_num}.npy',
            f'y_train_{dataset_id}_fold_{fold_num}.npy',
            f'X_val_{dataset_id}_fold_{fold_num}.npy',
            f'y_val_{dataset_id}_fold_{fold_num}.npy',
            f'X_test_{dataset_id}_fold_{fold_num}.npy',
            f'y_test_{dataset_id}_fold_{fold_num}.npy',
        ]
        
        fold_ok = True
        for filename in required_files:
            filepath = os.path.join(fold_files_directory, filename)
            if not os.path.exists(filepath):
                fold_ok = False
                all_folds_ok = False
                print(f"   ✗ Fold {fold_num}: Missing {filename}")
                break
        
        if fold_ok:
            print(f"   ✓ Fold {fold_num}: All files present")
    
    if not all_folds_ok:
        raise FileNotFoundError(
            f"\n❌ Some fold files are missing!\n"
            f"   Please ensure all 5 folds have complete data files.\n"
            f"   dataset_id: '{dataset_id}'\n"
        )
    
    print(f"\n✓ All 5 folds verified successfully!")
    
    # Load first fold to detect sequence_length
    print(f"\nLoading Fold 1 to detect sequence length...")
    x_train = np.load(os.path.join(fold_files_directory, f'X_train_{dataset_id}_fold_1.npy'))
    
    print(f"   Train samples: {x_train.shape}")
    
    # Update sequence length from actual data
    if len(x_train.shape) == 2:  # (samples, sequence_length)
        config['sequence_length'] = x_train.shape[1]
    elif len(x_train.shape) == 3:  # Already (samples, channels, sequence_length)
        config['sequence_length'] = x_train.shape[2]
    
    print(f"   Sequence length detected: {config['sequence_length']}")
    print(f"\n✓ Dataset verification complete!")
    print(f"   During evolution, each individual will train on all 5 folds.")
    print("="*60)

# Verify dataset availability
load_dataset(CONFIG)

print(f"\n{'='*60}")
print("DATASET READY FOR 5-FOLD CROSS-VALIDATION")
print(f"{'='*60}")
print(f"   Sequence length: {CONFIG['sequence_length']}")
print(f"   Input channels: {CONFIG['num_channels']}")
print(f"   Number of classes: {CONFIG['num_classes']}")
print(f"   Batch size: {CONFIG['batch_size']}")
print(f"   Audio classification task: Control (0) vs Pathological (1)")
print(f"\n   ⚠️ Each individual will be evaluated on ALL 5 folds")
print(f"   ⚠️ This makes evolution ~5x slower but much more robust")



## 4. Neural Network Architecture Definition

In [ ]:
class EvolvableCNN(nn.Module):
    """
    Evolvable CNN class for 1D audio processing.
    Uses Conv1D layers for audio/sequential data.
    """
    
    def __init__(self, genome: dict, config: dict):
        super(EvolvableCNN, self).__init__()
        self.genome = genome
        self.config = config
        
        # Build convolutional layers (1D for audio)
        self.conv_layers = self._build_conv_layers()
        
        # Calculate output size after convolutions
        self.conv_output_size = self._calculate_conv_output_size()
        
        # Build fully connected layers
        self.fc_layers = self._build_fc_layers()
        
    def _build_conv_layers(self) -> nn.ModuleList:
        """Builds 1D convolutional layers according to genome."""
        layers = nn.ModuleList()
        
        in_channels = self.config['num_channels']
        normalization_type = self.genome.get('normalization_type', 'layer')  # Default to layer for compatibility

        for i in range(self.genome['num_conv_layers']):
            out_channels = self.genome['filters'][i]
            kernel_size = self.genome['kernel_sizes'][i]
            
            # Ensure kernel size is odd and reasonable for 1D
            kernel_size = max(3, kernel_size if kernel_size % 2 == 1 else kernel_size + 1)
            padding = kernel_size // 2
            
            # 1D Convolutional layer
            conv = nn.Conv1d(in_channels, out_channels, kernel_size, padding=padding)
            layers.append(conv)
            
            # Normalization layer (Layer Normalization or Batch Normalization)
            if normalization_type == 'layer':
                # Layer Normalization: normaliza sobre features, no sobre batch
                # Para Conv1d output de shape (batch, channels, length), normalizamos los channels
                layers.append(nn.LayerNorm(out_channels))
            else:
                # Batch normalization (default)
                layers.append(nn.BatchNorm1d(out_channels))
            
            # Activation function
            activation_name = self.genome['activations'][i % len(self.genome['activations'])]
            activation_func = ACTIVATION_FUNCTIONS[activation_name]()
            layers.append(activation_func)
            
            # Max pooling (1D) - reduce sequence length
            pool_size = 2 if i < self.genome['num_conv_layers'] - 1 else 2
            layers.append(nn.MaxPool1d(pool_size, pool_size))
            
            # Optional dropout after pooling
            if i < self.genome['num_conv_layers'] - 1:
                layers.append(nn.Dropout(0.1))
            
            in_channels = out_channels
            
        return layers
    
    def _calculate_conv_output_size(self) -> int:
        """Calculates output size after convolutional layers."""
        # Create dummy tensor to calculate size
        dummy_input = torch.zeros(1, self.config['num_channels'], 
                                 self.config['sequence_length'])
        
        # Pass through convolutional layers
        x = dummy_input
        for layer in self.conv_layers:
            x = layer(x)
        
        # Flatten and get size
        return x.view(-1).shape[0]
    
    def _build_fc_layers(self) -> nn.ModuleList:
        """Builds fully connected layers."""
        layers = nn.ModuleList()
        
        input_size = self.conv_output_size
        normalization_type = self.genome.get('normalization_type', 'layer')  # Default to layer for compatibility

        for i in range(self.genome['num_fc_layers']):
            output_size = self.genome['fc_nodes'][i]
            
            # Linear layer
            layers.append(nn.Linear(input_size, output_size))
            
            # Normalization layer (Layer Normalization or Batch Normalization)
            if normalization_type == 'layer':
                # Layer Normalization for FC layers
                layers.append(nn.LayerNorm(output_size))
            else:
                # Batch normalization for FC layers (default)
                layers.append(nn.BatchNorm1d(output_size))
            
            # Activation
            layers.append(nn.ReLU())
            
            # Dropout if not last layer
            if i < self.genome['num_fc_layers'] - 1:
                layers.append(nn.Dropout(self.genome['dropout_rate']))
            
            input_size = output_size
        
        # Final classification layer
        layers.append(nn.Linear(input_size, self.config['num_classes']))
        
        return layers
    
    def forward(self, x):
        """Forward pass of the network."""
        # Ensure input is in correct format for Conv1d
        # Expected: (batch, channels, sequence_length)
        if len(x.shape) == 2:  # (batch, sequence)
            x = x.unsqueeze(1)  # Add channel dimension
        
        # Convolutional layers
        for layer in self.conv_layers:
            x = layer(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        for layer in self.fc_layers:
            x = layer(x)
        
        return x
    
    def get_architecture_summary(self) -> str:
        """Returns an architecture summary."""
        summary = []
        summary.append(f"Conv1D Layers: {self.genome['num_conv_layers']}")
        summary.append(f"Filters: {self.genome['filters']}")
        summary.append(f"Kernel Sizes: {self.genome['kernel_sizes']}")
        summary.append(f"FC Layers: {self.genome['num_fc_layers']}")
        summary.append(f"FC Nodes: {self.genome['fc_nodes']}")
        summary.append(f"Activations: {self.genome['activations']}")
        summary.append(f"Normalization: {self.genome.get('normalization_type', 'batch')}")
        summary.append(f"Dropout: {self.genome['dropout_rate']:.3f}")
        summary.append(f"Optimizer: {self.genome['optimizer']}")
        summary.append(f"Learning Rate: {self.genome['learning_rate']:.4f}")
        return " | ".join(summary)

print("EvolvableCNN class defined correctly (Conv1D for audio)")


## 5. Genetic Algorithm Components

In [ ]:
def create_random_genome(config: dict) -> dict:
    """Creates a random genome within specified ranges (optimized for 1D audio)."""
    # Number of layers
    num_conv_layers = random.randint(config['min_conv_layers'], config['max_conv_layers'])
    num_fc_layers = random.randint(config['min_fc_layers'], config['max_fc_layers'])

    # Filters for each convolutional layer (progressive increase)
    filters = []
    base_filters = random.randint(config['min_filters'], config['min_filters'] * 2)
    for i in range(num_conv_layers):
        # Gradually increase filters in deeper layers
        layer_filters = min(base_filters * (2 ** i), config['max_filters'])
        filters.append(layer_filters)

    # Kernel sizes (appropriate for 1D audio)
    kernel_sizes = [random.choice([3, 5, 7, 9, 11]) for _ in range(num_conv_layers)]

    # Nodes in fully connected layers (progressive decrease)
    fc_nodes = []
    base_fc = random.randint(config['min_fc_nodes'], config['max_fc_nodes'])
    for i in range(num_fc_layers):
        layer_nodes = max(config['min_fc_nodes'], base_fc // (2 ** i))
        fc_nodes.append(layer_nodes)

    # Activation functions for each layer
    activations = [random.choice(list(ACTIVATION_FUNCTIONS.keys())) for _ in range(max(num_conv_layers, num_fc_layers))]

    # Other parameters (adjusted for audio)
    dropout_rate = random.uniform(0.2, 0.5)
    learning_rate = random.choice([0.001, 0.0005, 0.0001, 0.00005])
    optimizer = random.choice(list(OPTIMIZERS.keys()))
    normalization_type = random.choice(['batch', 'layer'])  # Evolve normalization type

    genome = {
        'num_conv_layers': num_conv_layers,
        'num_fc_layers': num_fc_layers,
        'filters': filters,
        'kernel_sizes': kernel_sizes,
        'fc_nodes': fc_nodes,
        'activations': activations,
        'dropout_rate': dropout_rate,
        'learning_rate': learning_rate,
        'optimizer': optimizer,
        'normalization_type': normalization_type,
        'fitness': 0.0,
        'id': str(uuid.uuid4())[:8]
    }
    return genome

def mutate_genome(genome: dict, config: dict) -> dict:
    """Applies mutation to a genome using adaptive mutation rate."""
    mutated_genome = copy.deepcopy(genome)
    mutation_rate = config['current_mutation_rate']  # adaptive

    # Mutate number of convolutional layers
    if random.random() < mutation_rate:
        mutated_genome['num_conv_layers'] = random.randint(config['min_conv_layers'], config['max_conv_layers'])
        num_conv = mutated_genome['num_conv_layers']
        mutated_genome['filters'] = mutated_genome['filters'][:num_conv]
        mutated_genome['kernel_sizes'] = mutated_genome['kernel_sizes'][:num_conv]
        while len(mutated_genome['filters']) < num_conv:
            mutated_genome['filters'].append(random.randint(config['min_filters'], config['max_filters']))
        while len(mutated_genome['kernel_sizes']) < num_conv:
            mutated_genome['kernel_sizes'].append(random.choice([3, 5, 7, 9, 11]))

    # Mutate filters
    for i in range(len(mutated_genome['filters'])):
        if random.random() < mutation_rate:
            mutated_genome['filters'][i] = random.randint(config['min_filters'], config['max_filters'])

    # Mutate kernel sizes (1D appropriate sizes)
    for i in range(len(mutated_genome['kernel_sizes'])):
        if random.random() < mutation_rate:
            mutated_genome['kernel_sizes'][i] = random.choice([3, 5, 7, 9, 11, 13, 15])

    # Mutate number of FC layers
    if random.random() < mutation_rate:
        mutated_genome['num_fc_layers'] = random.randint(config['min_fc_layers'], config['max_fc_layers'])
        num_fc = mutated_genome['num_fc_layers']
        mutated_genome['fc_nodes'] = mutated_genome['fc_nodes'][:num_fc]
        while len(mutated_genome['fc_nodes']) < num_fc:
            mutated_genome['fc_nodes'].append(random.randint(config['min_fc_nodes'], config['max_fc_nodes']))

    # Mutate FC nodes
    for i in range(len(mutated_genome['fc_nodes'])):
        if random.random() < mutation_rate:
            mutated_genome['fc_nodes'][i] = random.randint(config['min_fc_nodes'], config['max_fc_nodes'])

    # Mutate activation functions
    for i in range(len(mutated_genome['activations'])):
        if random.random() < mutation_rate:
            mutated_genome['activations'][i] = random.choice(list(ACTIVATION_FUNCTIONS.keys()))

    # Mutate dropout
    if random.random() < mutation_rate:
        mutated_genome['dropout_rate'] = random.uniform(0.2, 0.6)

    # Mutate learning rate (audio-specific range)
    if random.random() < mutation_rate:
        mutated_genome['learning_rate'] = random.choice([0.001, 0.0005, 0.0001, 0.00005, 0.00001])

    # Mutate optimizer
    if random.random() < mutation_rate:
        mutated_genome['optimizer'] = random.choice(list(OPTIMIZERS.keys()))

    # Mutate normalization type (batch or layer)
    if random.random() < mutation_rate:
        mutated_genome['normalization_type'] = random.choice(['batch', 'layer'])

    mutated_genome['id'] = str(uuid.uuid4())[:8]
    mutated_genome['fitness'] = 0.0
    return mutated_genome

def crossover_genomes(parent1: dict, parent2: dict, config: dict) -> Tuple[dict, dict]:
    """Performs crossover between two genomes."""
    if random.random() > config['crossover_rate']:
        return copy.deepcopy(parent1), copy.deepcopy(parent2)

    child1 = copy.deepcopy(parent1)
    child2 = copy.deepcopy(parent2)

    # Crossover scalar parameters
    for key in ['num_conv_layers', 'num_fc_layers', 'dropout_rate', 'learning_rate', 'optimizer', 'normalization_type']:
        if random.random() < 0.5:
            child1[key], child2[key] = child2[key], child1[key]

    # Crossover lists (random cut point)
    for list_key in ['filters', 'kernel_sizes', 'fc_nodes', 'activations']:
        if random.random() < 0.5:
            list1 = child1[list_key]
            list2 = child2[list_key]
            if len(list1) > 1 and len(list2) > 1:
                point1 = random.randint(1, len(list1) - 1)
                point2 = random.randint(1, len(list2) - 1)
                child1[list_key] = list1[:point1] + list2[point2:]
                child2[list_key] = list2[:point2] + list1[point1:]

    child1['id'] = str(uuid.uuid4())[:8]
    child2['id'] = str(uuid.uuid4())[:8]
    child1['fitness'] = 0.0
    child2['fitness'] = 0.0
    return child1, child2

print("Genetic functions updated for adaptive mutation and 1D audio processing")


## 6. Hybrid Neuroevolution Implementation

In [ ]:
class HybridNeuroevolution:
    """Main class that implements hybrid neuroevolution with 5-fold CV and adaptive mutation."""

    def __init__(self, config: dict):
        self.config = config
        self.population = []
        self.generation = 0
        self.best_individual = None
        self.fitness_history = []
        self.generation_stats = []
        self.best_checkpoint_path = None  # Ruta del checkpoint del mejor modelo
        
        # Early stopping configuration at generation level
        self.generations_without_improvement = 0
        self.best_fitness_overall = -float('inf')
        self.min_improvement_threshold = 0.1  # Mínima mejora en fitness (%) para resetear contador
        self.max_generations_without_improvement = config.get('early_stopping_generations', 10)

    def initialize_population(self):
        print(f"Initializing population of {self.config['population_size']} individuals...")
        self.population = [create_random_genome(self.config) for _ in range(self.config['population_size'])]
        print(f"Population initialized with {len(self.population)} individuals")
    
    def save_best_checkpoint(self, genome: dict, model: nn.Module):
        """
        Guarda el checkpoint del mejor modelo global y elimina el anterior.
        
        Args:
            genome: Genoma del mejor modelo
            model: Modelo de PyTorch a guardar
        """
        # Crear directorio para checkpoints si no existe
        checkpoint_dir = "checkpoints"
        os.makedirs(checkpoint_dir, exist_ok=True)
        
        # Eliminar checkpoint anterior si existe
        if self.best_checkpoint_path and os.path.exists(self.best_checkpoint_path):
            try:
                os.remove(self.best_checkpoint_path)
                print(f"      ✓ Checkpoint anterior eliminado: {self.best_checkpoint_path}")
            except Exception as e:
                print(f"      ✗ Error eliminando checkpoint anterior: {e}")
        
        # Crear nuevo checkpoint
        checkpoint_filename = f"best_model_gen{self.generation}_id{genome['id']}_fitness{genome['fitness']:.2f}.pth"
        checkpoint_path = os.path.join(checkpoint_dir, checkpoint_filename)
        
        # Guardar modelo y genoma
        checkpoint_data = {
            'model_state_dict': model.state_dict(),
            'genome': genome,
            'generation': self.generation,
            'fitness': genome['fitness'],
            'config': self.config
        }
        
        try:
            torch.save(checkpoint_data, checkpoint_path)
            self.best_checkpoint_path = checkpoint_path
            print(f"      ✓ Nuevo checkpoint guardado: {checkpoint_path}")
            print(f"        Fitness: {genome['fitness']:.2f}%, ID: {genome['id']}, Gen: {self.generation}")
        except Exception as e:
            print(f"      ✗ Error guardando checkpoint: {e}")
    
    def load_best_checkpoint(self):
        """
        Carga el mejor checkpoint guardado.
        
        Returns:
            Tuple de (genome, model) o (None, None) si no hay checkpoint
        """
        if not self.best_checkpoint_path or not os.path.exists(self.best_checkpoint_path):
            print("No hay checkpoint disponible para cargar")
            return None, None
        
        try:
            checkpoint_data = torch.load(self.best_checkpoint_path, map_location=device)
            genome = checkpoint_data['genome']
            
            # Crear modelo y cargar pesos
            model = EvolvableCNN(genome, self.config).to(device)
            model.load_state_dict(checkpoint_data['model_state_dict'])
            
            print(f"✓ Checkpoint cargado exitosamente: {self.best_checkpoint_path}")
            print(f"  Fitness: {checkpoint_data['fitness']:.2f}%, Gen: {checkpoint_data['generation']}, ID: {genome['id']}")
            
            return genome, model
        except Exception as e:
            print(f"✗ Error cargando checkpoint: {e}")
            return None, None

    def _train_one_fold(self, model, optimizer, criterion, train_loader, test_loader, genome_id: str, fold_num: int):
        """
        Entrena y evalúa un modelo en un fold específico.
        
        Returns:
            float: Accuracy del fold
        """
        best_acc = 0.0
        best_epoch = -1
        patience_left = self.config['epoch_patience']
        last_improvement_acc = 0.0
        max_epochs = self.config['num_epochs']

        for epoch in range(1, max_epochs + 1):
            # Train
            model.train()
            running_loss = 0.0
            batch_count = 0
            max_batches = min(len(train_loader), self.config['early_stopping_patience'])
            
            for data, target in train_loader:
                data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                batch_count += 1
                if batch_count >= max_batches:
                    break
            
            avg_loss = running_loss / max(1, batch_count)
            
            # Evaluate
            model.eval()
            correct = 0
            total = 0
            eval_batches = 0
            max_eval_batches = min(len(test_loader), 20)
            total_eval_loss = 0.0
            
            with torch.no_grad():
                for data, target in test_loader:
                    data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)
                    output = model(data)
                    loss = criterion(output, target)
                    total_eval_loss += loss.item()
                    _, predicted = torch.max(output, 1)
                    total += target.size(0)
                    correct += (predicted == target).sum().item()
                    eval_batches += 1
                    if eval_batches >= max_eval_batches:
                        break
            
            acc = 100.0 * correct / max(1, total)
            avg_eval_loss = total_eval_loss / max(1, eval_batches)
            
            # Early stopping logic
            improvement = acc - last_improvement_acc
            if improvement >= self.config['improvement_threshold']:
                patience_left = self.config['epoch_patience']
                last_improvement_acc = acc
            else:
                patience_left -= 1

            if acc > best_acc:
                best_acc = acc
                best_epoch = epoch

            # Solo mostrar cada 5 épocas para no saturar el log
            if epoch % 5 == 0 or epoch == 1 or epoch == max_epochs:
                print(f"          Fold {fold_num} Epoch {epoch}: loss={avg_loss:.4f}, acc={acc:.2f}% (best={best_acc:.2f}%)")

            if patience_left <= 0:
                print(f"          Fold {fold_num}: Early stopping at epoch {epoch}")
                break
        
        return best_acc

    def _train_fold_in_thread(self, genome: dict, fold_num: int) -> Tuple[int, float, nn.Module]:
        """
        Entrena un modelo en un fold específico (diseñado para ejecutarse en un thread).
        
        Args:
            genome: Genoma del modelo
            fold_num: Número de fold (1-5)
        
        Returns:
            Tuple de (fold_num, accuracy, model)
        """
        try:
            # Cargar datos del fold
            fold_train_loader, fold_test_loader = self._load_fold_data(fold_num)
            
            # Crear nuevo modelo para este fold
            model = EvolvableCNN(genome, self.config).to(device)
            optimizer_class = OPTIMIZERS[genome['optimizer']]
            optimizer = optimizer_class(model.parameters(), lr=genome['learning_rate'])
            criterion = nn.CrossEntropyLoss()
            
            # Entrenar y evaluar en este fold
            fold_acc = self._train_one_fold(
                model, optimizer, criterion, 
                fold_train_loader, fold_test_loader,
                genome['id'], fold_num
            )
            
            print(f"      → Fold {fold_num} completed: {fold_acc:.2f}%")
            
            return fold_num, fold_acc, model
            
        except Exception as e:
            print(f"      ERROR in Fold {fold_num}: {e}")
            import traceback
            traceback.print_exc()
            return fold_num, 0.0, None

    def evaluate_fitness(self, genome: dict) -> tuple:
        """
        Evalúa el fitness de un genoma usando 5-fold cross-validation PARALELO.
        Los 5 folds se entrenan en threads separados y se espera a que terminen todos.
        El fitness final es el promedio de accuracy de los 5 folds.
        
        Returns:
            Tuple de (fitness, model) donde:
                - fitness: promedio de accuracies de los 5 folds
                - model: modelo entrenado en el mejor fold (para checkpoint)
        """
        print(f"      Training/Evaluating model {genome['id']} with PARALLEL 5-FOLD CROSS-VALIDATION")
        
        fold_accuracies = {}
        fold_models = {}
        
        try:
            # Usar ThreadPoolExecutor para ejecutar los 5 folds en paralelo
            with ThreadPoolExecutor(max_workers=5) as executor:
                # Enviar los 5 folds a threads separados
                print(f"      → Submitting 5 folds to thread pool...")
                futures = {
                    executor.submit(self._train_fold_in_thread, genome, fold_num): fold_num
                    for fold_num in range(1, 6)
                }
                
                # Esperar a que todos los folds terminen
                print(f"      → Waiting for all 5 folds to complete...")
                for future in as_completed(futures):
                    fold_num, fold_acc, model = future.result()
                    fold_accuracies[fold_num] = fold_acc
                    fold_models[fold_num] = model
            
            # Ordenar resultados por fold_num
            sorted_folds = sorted(fold_accuracies.keys())
            accuracies_list = [fold_accuracies[f] for f in sorted_folds]
            
            # Encontrar el mejor modelo
            best_fold_num = max(fold_accuracies, key=fold_accuracies.get)
            best_fold_acc = fold_accuracies[best_fold_num]
            best_model = fold_models[best_fold_num]
            
            # Calcular fitness como promedio de los 5 folds
            avg_fitness = np.mean(accuracies_list)
            std_fitness = np.std(accuracies_list)
            
            print(f"      ✓ PARALLEL 5-Fold CV Results for {genome['id']}:")
            print(f"        Fold accuracies: {[f'{acc:.2f}%' for acc in accuracies_list]}")
            print(f"        Average fitness: {avg_fitness:.2f}% ± {std_fitness:.2f}%")
            print(f"        Best fold: Fold {best_fold_num} with {best_fold_acc:.2f}%")
            
            return avg_fitness, best_model
            
        except Exception as e:
            print(f"      ERROR evaluating genome {genome['id']}: {e}")
            logger.warning(f"Error evaluating genome {genome['id']}: {e}")
            import traceback
            traceback.print_exc()
            return 0.0, None
    
    def _load_fold_data(self, fold_number: int):
        """
        Carga los datos de un fold específico para el entrenamiento.
        
        Args:
            fold_number: Número de fold (1-5)
        
        Returns:
            Tuple de (train_loader, test_loader)
        """
        fold_files_directory = os.path.join(
            self.config['data_path'], 
            f"files_{self.config['fold_id']}"
        )
        
        dataset_id = self.config['dataset_id']
        
        # Cargar datos del fold
        x_train = np.load(os.path.join(fold_files_directory, f'X_train_{dataset_id}_fold_{fold_number}.npy'))
        y_train = np.load(os.path.join(fold_files_directory, f'y_train_{dataset_id}_fold_{fold_number}.npy'))
        x_val = np.load(os.path.join(fold_files_directory, f'X_val_{dataset_id}_fold_{fold_number}.npy'))
        y_val = np.load(os.path.join(fold_files_directory, f'y_val_{dataset_id}_fold_{fold_number}.npy'))
        x_test = np.load(os.path.join(fold_files_directory, f'X_test_{dataset_id}_fold_{fold_number}.npy'))
        y_test = np.load(os.path.join(fold_files_directory, f'y_test_{dataset_id}_fold_{fold_number}.npy'))
        
        # Reshape si es necesario
        if len(x_train.shape) == 2:
            x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
            x_val = x_val.reshape((x_val.shape[0], 1, x_val.shape[1]))
            x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
        
        # Convertir a tensores
        x_train_tensor = torch.FloatTensor(x_train)
        y_train_tensor = torch.LongTensor(y_train.astype(np.int64))
        x_val_tensor = torch.FloatTensor(x_val)
        y_val_tensor = torch.LongTensor(y_val.astype(np.int64))
        x_test_tensor = torch.FloatTensor(x_test)
        y_test_tensor = torch.LongTensor(y_test.astype(np.int64))
        
        # Crear datasets
        train_dataset = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
        x_eval = torch.cat([x_val_tensor, x_test_tensor], dim=0)
        y_eval = torch.cat([y_val_tensor, y_test_tensor], dim=0)
        test_dataset = torch.utils.data.TensorDataset(x_eval, y_eval)
        
        # Crear DataLoaders
        fold_train_loader = DataLoader(
            train_dataset, 
            batch_size=self.config['batch_size'], 
            shuffle=True,
            num_workers=0,
            pin_memory=True if torch.cuda.is_available() else False
        )
        
        fold_test_loader = DataLoader(
            test_dataset, 
            batch_size=self.config['batch_size'], 
            shuffle=False,
            num_workers=0,
            pin_memory=True if torch.cuda.is_available() else False
        )
        
        return fold_train_loader, fold_test_loader

    def evaluate_population(self):
        print(f"\nEvaluating population (Generation {self.generation})...")
        print(f"Processing {len(self.population)} individuals...")
        fitness_scores = []
        best_fitness_so_far = 0.0
        current_global_best_fitness = self.best_individual['fitness'] if self.best_individual else 0.0
        
        for i, genome in enumerate(self.population):
            print(f"\n   Evaluating individual {i+1}/{len(self.population)} (ID: {genome['id']})")
            print(f"      Architecture: {genome['num_conv_layers']} conv + {genome['num_fc_layers']} fc, opt={genome['optimizer']}, lr={genome['learning_rate']}")
            
            # Evaluar y obtener fitness y modelo
            fitness, model = self.evaluate_fitness(genome)
            genome['fitness'] = fitness
            fitness_scores.append(fitness)
            
            if fitness > best_fitness_so_far:
                best_fitness_so_far = fitness
                print(f"      New best fitness in this generation: {fitness:.2f}%!")
            
            # Verificar si es un nuevo mejor global
            if fitness > current_global_best_fitness:
                print(f"      🌟 NEW GLOBAL BEST! {fitness:.2f}% > {current_global_best_fitness:.2f}%")
                current_global_best_fitness = fitness
                
                # Guardar checkpoint (elimina el anterior automáticamente)
                if model is not None:
                    self.save_best_checkpoint(genome, model)
            
            print(f"      Fitness obtained: {fitness:.2f}% | Best in generation: {best_fitness_so_far:.2f}% | Global best: {current_global_best_fitness:.2f}%")
        # Generation statistics
        if fitness_scores:
            avg_fitness = np.mean(fitness_scores)
            max_fitness = np.max(fitness_scores)
            min_fitness = np.min(fitness_scores)
            std_fitness = np.std(fitness_scores)
        else:
            avg_fitness = max_fitness = min_fitness = std_fitness = 0.0

        stats = {
            'generation': self.generation,
            'avg_fitness': avg_fitness,
            'max_fitness': max_fitness,
            'min_fitness': min_fitness,
            'std_fitness': std_fitness
        }
        self.generation_stats.append(stats)
        self.fitness_history.append(max_fitness)

        best_genome = max(self.population, key=lambda x: x['fitness'])
        if self.best_individual is None or best_genome['fitness'] > self.best_individual['fitness']:
            self.best_individual = copy.deepcopy(best_genome)
            print(f"\nNew global best individual found!")

        print(f"\nGENERATION {self.generation} STATISTICS:")
        print(f"   Maximum fitness: {max_fitness:.2f}%")
        print(f"   Average fitness: {avg_fitness:.2f}%")
        print(f"   Minimum fitness: {min_fitness:.2f}%")
        print(f"   Standard deviation: {std_fitness:.2f}%")
        print(f"   Best individual: {best_genome['id']} with {best_genome['fitness']:.2f}%")
        print(f"   Global best individual: {self.best_individual['id']} with {self.best_individual['fitness']:.2f}%")

    def selection_and_reproduction(self):
        print(f"\nStarting selection and reproduction...")
        # Sort by fitness
        self.population.sort(key=lambda x: x['fitness'], reverse=True)
        elite_size = max(1, int(self.config['population_size'] * self.config['elite_percentage']))
        elite = self.population[:elite_size]
        print(f"Selecting {elite_size} elite individuals:")
        for i, individual in enumerate(elite):
            print(f"   Elite {i+1}: {individual['id']} (fitness: {individual['fitness']:.2f}%)")
        new_population = copy.deepcopy(elite)
        offspring_needed = self.config['population_size'] - len(new_population)
        print(f"Creating {offspring_needed} new individuals through crossover and mutation...")
        offspring_created = 0
        while len(new_population) < self.config['population_size']:
            parent1 = self.tournament_selection()
            parent2 = self.tournament_selection()
            child1, child2 = crossover_genomes(parent1, parent2, self.config)
            child1 = mutate_genome(child1, self.config)
            if len(new_population) < self.config['population_size']:
                new_population.append(child1)
            child2 = mutate_genome(child2, self.config)
            if len(new_population) < self.config['population_size']:
                new_population.append(child2)
            offspring_created += 2
            if offspring_created % 4 == 0:
                print(f"   Created {min(offspring_created, offspring_needed)} of {offspring_needed} new individuals...")
        self.population = new_population[:self.config['population_size']]
        print(f"New generation created with {len(self.population)} individuals")
        print(f"   Elite preserved: {elite_size}")
        print(f"   New individuals: {len(self.population) - elite_size}")

    def tournament_selection(self, tournament_size: int = 3) -> dict:
        tournament = random.sample(self.population, min(tournament_size, len(self.population)))
        return max(tournament, key=lambda x: x['fitness'])

    def _update_adaptive_mutation(self):
        # Diversity measured via std of fitness in last generation
        if not self.generation_stats:
            self.config['current_mutation_rate'] = self.config['base_mutation_rate']
            return
        last_std = self.generation_stats[-1]['std_fitness']
        # Heuristic: more diversity -> lower mutation, low diversity -> higher
        # Normalize std roughly assuming fitness in [0,100]
        diversity_factor = min(1.0, last_std / 10.0)  # std 10% -> factor 1
        # Invert: low diversity (small std) should raise mutation
        inverted = 1 - diversity_factor
        new_rate = self.config['base_mutation_rate'] + (inverted - 0.5) * 0.4  # adjust +/-0.2 range
        new_rate = max(self.config['mutation_rate_min'], min(self.config['mutation_rate_max'], new_rate))
        self.config['current_mutation_rate'] = round(new_rate, 4)
        print(f"Adaptive mutation rate updated to {self.config['current_mutation_rate']} (std_fitness={last_std:.2f})")

    def check_convergence(self) -> bool:
        """
        Verifica criterios de convergencia:
        1. Target fitness alcanzado
        2. Máximo de generaciones alcanzado
        3. Early stopping: sin mejora en N generaciones
        4. Estancamiento detectado en últimas generaciones
        """
        # Criterion 1: Target fitness reached
        if self.best_individual and self.best_individual['fitness'] >= self.config['fitness_threshold']:
            print(f"\n✅ Target fitness reached! ({self.best_individual['fitness']:.2f}% >= {self.config['fitness_threshold']}%)")
            return True
        
        # Criterion 2: Maximum generations reached
        if self.generation >= self.config['max_generations']:
            print(f"\n⏱️ Maximum generations reached ({self.generation}/{self.config['max_generations']})")
            return True
        
        # Criterion 3: Early stopping - no improvement in N generations
        if self.generation > 0:  # No check on generation 0
            current_best = self.best_individual['fitness'] if self.best_individual else 0.0
            
            # Check if there's improvement compared to best overall
            improvement = current_best - self.best_fitness_overall
            
            if improvement >= self.min_improvement_threshold:
                # Significant improvement! Reset counter
                self.best_fitness_overall = current_best
                self.generations_without_improvement = 0
                print(f"\n🔄 Improvement detected: {improvement:.2f}% | Generations without improvement: {self.generations_without_improvement}")
            else:
                # No significant improvement
                self.generations_without_improvement += 1
                print(f"\n⏳ No significant improvement | Generations without improvement: {self.generations_without_improvement}/{self.max_generations_without_improvement}")
                
                if self.generations_without_improvement >= self.max_generations_without_improvement:
                    print(f"\n🛑 EARLY STOPPING: No improvement for {self.max_generations_without_improvement} generations")
                    print(f"   Best fitness plateau: {self.best_fitness_overall:.2f}%")
                    return True
        
        # Criterion 4: Stagnation in last 3 generations (additional safety check)
        if len(self.fitness_history) >= 3:
            recent = self.fitness_history[-3:]
            if max(recent) - min(recent) < 0.5:
                print(f"\n📉 Stagnation detected in last 3 generations (all within {max(recent) - min(recent):.2f}%)")
                # Don't stop immediately, let generation-level early stopping handle it
        
        return False

    def evolve(self) -> dict:
        print("STARTING HYBRID NEUROEVOLUTION PROCESS (adaptive mutation + generation-level early stopping)")
        print("="*80)
        print(f"Configuration:")
        print(f"   Population: {self.config['population_size']} individuals")
        print(f"   Maximum generations: {self.config['max_generations']}")
        print(f"   Target fitness: {self.config['fitness_threshold']}%")
        print(f"   Early stopping (generations): {self.config['early_stopping_generations']} without improvement")
        print(f"   Min improvement threshold: {self.config['min_improvement_threshold']}%")
        print(f"   Device: {device}")
        print("="*80)
        self.initialize_population()
        while not self.check_convergence():
            print(f"\n{'='*80}")
            print(f"GENERATION {self.generation}")
            print(f"{'='*80}")
            self.evaluate_population()
            if self.check_convergence():
                break
            self._update_adaptive_mutation()
            self.selection_and_reproduction()
            self.generation += 1
            print(f"\nPreparing for next generation...")
        print(f"\n{'='*80}")
        print(f"EVOLUTION COMPLETED!")
        print(f"{'='*80}")
        print(f"Best individual found:")
        print(f"   ID: {self.best_individual['id']}")
        print(f"   Fitness: {self.best_individual['fitness']:.2f}%")
        print(f"   Origin generation: {self.generation}")
        print(f"   Total generations processed: {self.generation + 1}")
        print(f"   Generations without improvement: {self.generations_without_improvement}/{self.max_generations_without_improvement}")
        print("="*80)
        return self.best_individual

print("HybridNeuroevolution class updated with PARALLEL 5-fold CV evaluation and adaptive mutation")

## 7. Evolution Process Execution

### 🚀 Importante: Parallel 5-Fold Cross-Validation durante la Evolución

**Cambio clave**: Ahora cada individuo se evalúa con **5-fold cross-validation PARALELO** durante el proceso evolutivo:

1. **Durante la evolución**:
   - Cada individuo se entrena y evalúa en **cada uno de los 5 folds SIMULTÁNEAMENTE**
   - Los 5 folds se ejecutan en **threads separados** (paralelización)
   - El **fitness final** es el **promedio** de las accuracies de los 5 folds
   - Se espera a que **todos los folds terminen** antes de calcular el fitness
   - Esto garantiza que la arquitectura seleccionada no esté sobreajustada a un fold específico

2. **Ventajas de la paralelización**:
   - 🚀 **Mucho más rápido**: Los 5 folds se entrenan simultáneamente (en threads)
   - ✅ **Más robusto**: La mejor arquitectura generaliza mejor
   - ✅ **Menos sesgado**: No depende de un solo split de datos
   - 💡 **Aprovecha multi-core**: Usa múltiples núcleos de CPU para acelerar
   
3. **Proceso paralelo**:
   - Generación 0: Se crean N individuos aleatorios
   - Para cada individuo:
     - **Thread Pool**: Se crean 5 threads (uno por fold)
     - **Fold 1-5**: Se entrenan y evalúan **SIMULTÁNEAMENTE** → accuracy₁...accuracy₅
     - **Espera**: Se espera a que **todos los threads terminen**
     - **Fitness** = (accuracy₁ + accuracy₂ + accuracy₃ + accuracy₄ + accuracy₅) / 5
   - Se seleccionan los mejores según fitness promedio
   - Se aplica crossover y mutación
   - Siguiente generación...

4. **Rendimiento**:
   - Tiempo de evaluación ≈ tiempo del fold más lento (en lugar de suma de todos)
   - Aceleración teórica: ~5x más rápido que secuencial
   - Aceleración real: depende del número de cores disponibles

**Nota**: Para hacer pruebas rápidas, puedes reducir `population_size` y `max_generations` en la configuración.

In [ ]:
# ==========================================
# CONFIGURACIÓN DE DATASET DE AUDIO
# ==========================================

# Ruta OS-independiente usando os.path.join
CONFIG['data_path'] = os.path.join('data', 'sets', 'folds_5')

# ==========================================
# AJUSTES OPCIONALES
# ==========================================

# Ajustar población y generaciones si es necesario
# CONFIG['population_size'] = 8
# CONFIG['max_generations'] = 20
# CONFIG['fitness_threshold'] = 85.0  # Para audio, 85% es buen objetivo
# CONFIG['batch_size'] = 16  # Reducir si hay problemas de memoria

print("="*60)
print("AUDIO NEUROEVOLUTION CONFIGURATION")
print("="*60)
print(f"   Dataset: Audio (Parkinson Classification)")
print(f"   Dataset ID: {CONFIG['dataset_id']}")
print(f"   Fold ID: {CONFIG['fold_id']}")
print(f"   Number of folds: {CONFIG['num_folds']} (all used during evolution)")
print(f"   Data Path: {CONFIG['data_path']}")
print(f"   Number of channels: {CONFIG['num_channels']} (1D audio)")
print(f"   Sequence length: {CONFIG['sequence_length']} (will be auto-detected)")
print(f"   Number of classes: {CONFIG['num_classes']} (Control vs Pathological)")
print(f"   Batch size: {CONFIG['batch_size']}")
print(f"   Population: {CONFIG['population_size']} individuals")
print(f"   Maximum generations: {CONFIG['max_generations']}")
print(f"   Target fitness: {CONFIG['fitness_threshold']}%")
print(f"   Device: {device}")
print(f"   Platform: {os.name} ({'Windows' if os.name == 'nt' else 'Unix/Linux/Mac'})")
print(f"   Parallelization: Enabled (5 threads per individual)")
print("="*60)

# Verify dataset availability with the new configuration
print(f"\nVerifying audio dataset...")
load_dataset(CONFIG)

print(f"\n{'='*60}")
print("DATASET VERIFIED - READY FOR PARALLEL 5-FOLD CV EVOLUTION")
print(f"{'='*60}")

# Initialize neuroevolution system
start_time = datetime.now()
print(f"\nStarting audio neuroevolution at {start_time.strftime('%H:%M:%S')}")
print(f"Architecture: Conv1D -> BatchNorm1D -> Activation -> MaxPool1D -> FC")
print(f"Each individual will be evaluated on all 5 folds IN PARALLEL")
print(f"Using ThreadPoolExecutor with 5 workers (one per fold)")
print(f"{'='*60}\n")

# Create system instance (no need for train/test loaders anymore)
neuroevolution = HybridNeuroevolution(CONFIG)

# Execute evolution process
best_genome = neuroevolution.evolve()

end_time = datetime.now()
execution_time = end_time - start_time

print(f"\n{'='*60}")
print("EVOLUTION PROCESS COMPLETED")
print(f"{'='*60}")
print(f"Completed at: {end_time.strftime('%H:%M:%S')}")
print(f"Total execution time: {execution_time}")
print(f"Total generations: {neuroevolution.generation}")
print(f"Best fitness achieved: {best_genome['fitness']:.2f}%")
print(f"{'='*60}")


## 8. Results Visualization and Analysis

In [ ]:
# Configure matplotlib style
plt.style.use('default')
sns.set_palette("husl")

# Function to visualize fitness evolution
def plot_fitness_evolution(neuroevolution):
    """Plots fitness evolution across generations."""
    if not neuroevolution.generation_stats:
        print("WARNING: No statistics data to plot")
        return
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Extract data and filter 0.00 fitness
    generations = []
    avg_fitness = []
    max_fitness = []
    min_fitness = []
    std_fitness = []
    
    for stat in neuroevolution.generation_stats:
        # Only include if valid fitness (> 0.00)
        if stat['max_fitness'] > 0.00:
            generations.append(stat['generation'])
            avg_fitness.append(stat['avg_fitness'])
            max_fitness.append(stat['max_fitness'])
            min_fitness.append(stat['min_fitness'])
            std_fitness.append(stat['std_fitness'])
    
    if not generations:
        print("WARNING: No valid fitness data to plot (all are 0.00)")
        return
    
    # Graph 1: Fitness evolution
    ax1.plot(generations, max_fitness, 'g-', linewidth=2, marker='o', label='Maximum Fitness')
    ax1.plot(generations, avg_fitness, 'b-', linewidth=2, marker='s', label='Average Fitness')
    ax1.plot(generations, min_fitness, 'r-', linewidth=2, marker='^', label='Minimum Fitness')
    ax1.fill_between(generations, 
                     [max(0, avg - std) for avg, std in zip(avg_fitness, std_fitness)],
                     [avg + std for avg, std in zip(avg_fitness, std_fitness)],
                     alpha=0.2, color='blue')
    
    ax1.set_xlabel('Generation')
    ax1.set_ylabel('Fitness (%)')
    ax1.set_title('Fitness Evolution by Generation (Excluding 0.00%)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Add target fitness line
    ax1.axhline(y=CONFIG['fitness_threshold'], color='orange', linestyle='--', 
                label=f"Target ({CONFIG['fitness_threshold']}%)")
    ax1.legend()
    
    # Set Y axis limits for better visualization
    y_min = max(0, min(min_fitness) - 5)
    y_max = min(100, max(max_fitness) + 5)
    ax1.set_ylim(y_min, y_max)
    
    # Graph 2: Diversity (standard deviation)
    ax2.plot(generations, std_fitness, 'purple', linewidth=2, marker='D')
    ax2.set_xlabel('Generation')
    ax2.set_ylabel('Fitness Standard Deviation')
    ax2.set_title('Population Diversity (Excluding 0.00%)')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Show additional information
    print(f"Plotted data:")
    print(f"   Generations with valid fitness: {len(generations)}")
    print(f"   Best fitness achieved: {max(max_fitness):.2f}%")
    print(f"   Final average fitness: {avg_fitness[-1]:.2f}%")
    if len(generations) < len(neuroevolution.generation_stats):
        excluded = len(neuroevolution.generation_stats) - len(generations)
        print(f"   WARNING: Excluded generations (0.00 fitness): {excluded}")

# Function to show detailed statistics
def show_evolution_statistics(neuroevolution):
    """Shows detailed evolution statistics."""
    print("DETAILED EVOLUTION STATISTICS")
    print("="*60)
    
    if not neuroevolution.generation_stats:
        print("WARNING: No statistics available")
        return
    
    # Filter statistics with valid fitness
    valid_stats = [stat for stat in neuroevolution.generation_stats if stat['max_fitness'] > 0.00]
    
    if not valid_stats:
        print("WARNING: No valid statistics (all fitness are 0.00)")
        return
    
    final_stats = valid_stats[-1]
    
    print(f"Completed generations: {neuroevolution.generation}")
    print(f"Generations with valid fitness: {len(valid_stats)}")
    if len(valid_stats) < len(neuroevolution.generation_stats):
        excluded = len(neuroevolution.generation_stats) - len(valid_stats)
        print(f"WARNING: Generations with 0.00 fitness (excluded): {excluded}")
    
    print(f"\nFINAL STATISTICS (excluding 0.00 fitness):")
    print(f"   Final best fitness: {final_stats['max_fitness']:.2f}%")
    print(f"   Final average fitness: {final_stats['avg_fitness']:.2f}%")
    print(f"   Final minimum fitness: {final_stats['min_fitness']:.2f}%")
    print(f"   Final standard deviation: {final_stats['std_fitness']:.2f}%")
    
    # Progress across generations
    if len(valid_stats) > 1:
        initial_max = valid_stats[0]['max_fitness']
        final_max = valid_stats[-1]['max_fitness']
        improvement = final_max - initial_max
        
        print(f"\nPROGRESS:")
        print(f"   Initial fitness: {initial_max:.2f}%")
        print(f"   Final fitness: {final_max:.2f}%")
        print(f"   Total improvement: {improvement:.2f}%")
        if initial_max > 0:
            print(f"   Relative improvement: {(improvement/initial_max)*100:.1f}%")
    
    # Convergence analysis
    print(f"\nCONVERGENCE CRITERIA:")
    if neuroevolution.best_individual and neuroevolution.best_individual['fitness'] >= CONFIG['fitness_threshold']:
        print(f"   OK: Target fitness reached ({CONFIG['fitness_threshold']}%)")
    else:
        print(f"   ERROR: Target fitness NOT reached ({CONFIG['fitness_threshold']}%)")
    
    if neuroevolution.generation >= CONFIG['max_generations']:
        print(f"   TIME: Maximum generations reached ({CONFIG['max_generations']})")
    
    # Additional performance statistics
    all_max_fitness = [stat['max_fitness'] for stat in valid_stats]
    all_avg_fitness = [stat['avg_fitness'] for stat in valid_stats]
    
    print(f"\nGENERAL STATISTICS:")
    print(f"   Best fitness of entire evolution: {max(all_max_fitness):.2f}%")
    print(f"   Average fitness of entire evolution: {np.mean(all_avg_fitness):.2f}%")
    print(f"   Average improvement per generation: {(max(all_max_fitness) - min(all_max_fitness))/len(valid_stats):.2f}%")
    
    if neuroevolution.best_individual:
        print(f"\nBest individual ID: {neuroevolution.best_individual['id']}")
        print(f"Best individual fitness: {neuroevolution.best_individual['fitness']:.2f}%")

# Additional function for failure analysis
def analyze_failed_evaluations(neuroevolution):
    """Analyzes evaluations that resulted in 0.00 fitness."""
    print("\nFAILED EVALUATIONS ANALYSIS")
    print("="*50)
    
    total_generations = len(neuroevolution.generation_stats)
    failed_generations = len([stat for stat in neuroevolution.generation_stats if stat['max_fitness'] == 0.00])
    
    if failed_generations == 0:
        print("OK: No failed evaluations (0.00 fitness)")
        return
    
    success_rate = ((total_generations - failed_generations) / total_generations) * 100
    
    print(f"Failure summary:")
    print(f"   Total generations: {total_generations}")
    print(f"   Failed generations: {failed_generations}")
    print(f"   Success rate: {success_rate:.1f}%")
    
    if failed_generations > 0:
        failed_gens = [stat['generation'] for stat in neuroevolution.generation_stats if stat['max_fitness'] == 0.00]
        print(f"   Generations with failures: {failed_gens}")
        
        print(f"\nPossible causes of 0.00 fitness:")
        print(f"   • Errors in model architecture")
        print(f"   • Memory problems (GPU/RAM)")
        print(f"   • Invalid hyperparameter configurations")
        print(f"   • Errors during training")

# Execute visualizations
plot_fitness_evolution(neuroevolution)
show_evolution_statistics(neuroevolution)
analyze_failed_evaluations(neuroevolution)

## 9. BEST ARCHITECTURE FOUND

In [ ]:
def display_best_architecture(best_genome, config):
    """
    Shows the best architecture found in detailed and visual format.
    """
    print("="*60)
    print("    BEST EVOLVED ARCHITECTURE (1D AUDIO)")
    print("="*60)
    
    # General information
    print(f"\nGENERAL INFORMATION:")
    print(f"   Genome ID: {best_genome['id']}")
    print(f"   Fitness Achieved: {best_genome['fitness']:.2f}%")
    print(f"   Generation: {neuroevolution.generation}")
    print(f"   Dataset: {config['dataset']}")
    print(f"   Dataset ID: {config.get('dataset_id', 'N/A')}")
    print(f"   Fold: {config.get('current_fold', 'N/A')}")
    
    # Architecture details
    print(f"\nNETWORK ARCHITECTURE:")
    print(f"   Input: 1D Audio Signal (length={config['sequence_length']})")
    print(f"   Convolutional Layers (Conv1D): {best_genome['num_conv_layers']}")
    print(f"   Fully Connected Layers: {best_genome['num_fc_layers']}")
    print(f"   Output: {config['num_classes']} classes")
    
    print(f"\nCONVOLUTIONAL LAYER DETAILS (1D):")
    for i in range(best_genome['num_conv_layers']):
        filters = best_genome['filters'][i]
        kernel = best_genome['kernel_sizes'][i]
        activation = best_genome['activations'][i % len(best_genome['activations'])]
        print(f"   Conv1D-{i+1}: {filters} filters, kernel_size={kernel}, activation={activation}")
        print(f"             -> BatchNorm1D -> {activation.upper()} -> MaxPool1D(2)")
    
    print(f"\nFULLY CONNECTED LAYER DETAILS:")
    for i, nodes in enumerate(best_genome['fc_nodes']):
        print(f"   FC{i+1}: {nodes} neurons -> BatchNorm1D -> ReLU -> Dropout({best_genome['dropout_rate']:.3f})")
    print(f"   Output: {config['num_classes']} neurons (Control vs Pathological)")
    
    print(f"\nHYPERPARAMETERS:")
    print(f"   Optimizer: {best_genome['optimizer'].upper()}")
    print(f"   Learning Rate: {best_genome['learning_rate']:.6f}")
    print(f"   Dropout Rate: {best_genome['dropout_rate']:.3f}")
    print(f"   Activation Functions: {', '.join(set(best_genome['activations']))}")
    
    # Create and show final model
    print(f"\nCREATING FINAL MODEL...")
    try:
        final_model = EvolvableCNN(best_genome, config)
        total_params = sum(p.numel() for p in final_model.parameters())
        trainable_params = sum(p.numel() for p in final_model.parameters() if p.requires_grad)
        
        print(f"   Model created successfully")
        print(f"   Total parameters: {total_params:,}")
        print(f"   Trainable parameters: {trainable_params:,}")
        print(f"   Model size: ~{total_params * 4 / 1024 / 1024:.2f} MB (float32)")
        
        # Architecture summary
        print(f"\nCOMPACT SUMMARY:")
        print(f"   {final_model.get_architecture_summary()}")
        
    except Exception as e:
        print(f"   ERROR creating model: {e}")
    
    # Visualization in table format
    print(f"\nSUMMARY TABLE:")
    print(f"{'='*80}")
    print(f"{'Parameter':<25} {'Value':<30} {'Description':<25}")
    print(f"{'='*80}")
    print(f"{'ID':<25} {best_genome['id']:<30} {'Unique identifier':<25}")
    print(f"{'Fitness':<25} {best_genome['fitness']:.2f}%{'':<25} {'Accuracy achieved':<25}")
    print(f"{'Architecture':<25} {'Conv1D + FC':<30} {'1D Convolutional':<25}")
    print(f"{'Conv Layers':<25} {best_genome['num_conv_layers']:<30} {'Conv1D layers':<25}")
    print(f"{'FC Layers':<25} {best_genome['num_fc_layers']:<30} {'FC layers':<25}")
    print(f"{'Optimizer':<25} {best_genome['optimizer']:<30} {'Optimization algorithm':<25}")
    print(f"{'Learning Rate':<25} {best_genome['learning_rate']:<30.6f} {'Learning rate':<25}")
    print(f"{'Dropout':<25} {best_genome['dropout_rate']:<30} {'Dropout rate':<25}")
    print(f"{'Input Length':<25} {config['sequence_length']:<30} {'Audio sequence length':<25}")
    print(f"{'Classes':<25} {config['num_classes']:<30} {'Binary classification':<25}")
    print(f"{'='*80}")
    
    # Comparison with initial configuration
    print(f"\nCOMPARISON WITH OBJECTIVES:")
    if best_genome['fitness'] >= config['fitness_threshold']:
        print(f"   ✓ TARGET REACHED: {best_genome['fitness']:.2f}% >= {config['fitness_threshold']}%")
    else:
        print(f"   ✗ TARGET NOT REACHED: {best_genome['fitness']:.2f}% < {config['fitness_threshold']}%")
        print(f"     Gap: {config['fitness_threshold'] - best_genome['fitness']:.2f}%")
    
    print(f"   Generations used: {neuroevolution.generation}/{config['max_generations']}")
    
    # Save information to JSON
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_file = f"best_architecture_audio_{timestamp}.json"
    
    results_data = {
        'timestamp': timestamp,
        'execution_time': str(execution_time),
        'dataset_type': 'audio_1D',
        'dataset_id': config.get('dataset_id', 'N/A'),
        'fold': config.get('current_fold', 'N/A'),
        'config_used': {k: v for k, v in config.items() if not k.startswith('_')},
        'best_genome': best_genome,
        'final_generation': neuroevolution.generation,
        'evolution_stats': neuroevolution.generation_stats
    }
    
    try:
        with open(results_file, 'w') as f:
            json.dump(results_data, f, indent=2, default=str)
        print(f"\n✓ Results saved to: {results_file}")
    except Exception as e:
        print(f"\n✗ WARNING: Error saving results: {e}")
    
    print(f"\n{'='*60}")
    print("HYBRID NEUROEVOLUTION FOR AUDIO COMPLETED!")
    print(f"{'='*60}")

# Show the best architecture found
display_best_architecture(best_genome, CONFIG)


In [ ]:
# Verificar información del checkpoint guardado
print("="*80)
print("INFORMACIÓN DEL CHECKPOINT DEL MEJOR MODELO")
print("="*80)

if neuroevolution.best_checkpoint_path:
    print(f"\n✓ Checkpoint guardado en: {neuroevolution.best_checkpoint_path}")
    
    # Obtener información del archivo
    import os
    if os.path.exists(neuroevolution.best_checkpoint_path):
        file_size = os.path.getsize(neuroevolution.best_checkpoint_path)
        file_size_mb = file_size / (1024 * 1024)
        print(f"  Tamaño: {file_size_mb:.2f} MB")
        
        # Cargar y mostrar información del checkpoint
        checkpoint_data = torch.load(neuroevolution.best_checkpoint_path, map_location=device)
        print(f"\n  Información del modelo guardado:")
        print(f"    Generación: {checkpoint_data['generation']}")
        print(f"    Fitness: {checkpoint_data['fitness']:.2f}%")
        print(f"    ID Genoma: {checkpoint_data['genome']['id']}")
        print(f"    Arquitectura: {checkpoint_data['genome']['num_conv_layers']} Conv1D + {checkpoint_data['genome']['num_fc_layers']} FC")
        print(f"    Optimizador: {checkpoint_data['genome']['optimizer']}")
        print(f"    Learning Rate: {checkpoint_data['genome']['learning_rate']}")
        
        print(f"\n  Este checkpoint se usará como punto de partida para el 5-fold CV")
        print(f"  (Transfer learning desde el modelo pre-entrenado)")
    else:
        print(f"  ✗ Archivo no encontrado")
else:
    print("\n✗ No hay checkpoint disponible")
    print("  El 5-fold CV entrenará desde cero")

print("\n" + "="*80)

---

## 🔄 Resumen del Flujo con Checkpoints

```
PROCESO DE NEUROEVOLUCIÓN CON CHECKPOINTS:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. EVOLUCIÓN (Múltiples Generaciones)
   │
   ├─ Para cada individuo:
   │  ├─ Entrenar y evaluar
   │  ├─ Calcular fitness
   │  │
   │  └─ SI fitness > mejor_global:
   │     ├─ 🌟 NUEVO MEJOR GLOBAL
   │     ├─ ✗ Eliminar checkpoint anterior
   │     └─ ✓ Guardar nuevo checkpoint
   │
   └─ Continuar hasta convergencia

2. AL FINALIZAR LA EVOLUCIÓN
   │
   └─ Se tiene el checkpoint del MEJOR modelo global

3. EVALUACIÓN 5-FOLD CROSS-VALIDATION
   │
   ├─ ✓ Cargar checkpoint del mejor modelo
   │
   ├─ Para cada fold (1 a 5):
   │  ├─ Crear modelo nuevo
   │  ├─ Inicializar con pesos pre-entrenados (Transfer Learning)
   │  ├─ Fine-tuning con datos del fold
   │  └─ Evaluar y guardar métricas
   │
   └─ Calcular promedios y desviaciones estándar

4. RESULTADOS FINALES
   └─ Métricas robustas para la tabla de comparación
```

### ✨ Beneficios de este enfoque:

- ✅ **Ahorro de espacio**: Solo 1 checkpoint (el mejor)
- ✅ **Eficiencia**: Transfer learning en lugar de entrenar desde cero
- ✅ **Robustez**: Métricas con 5-fold CV
- ✅ **Trazabilidad**: Se mantiene el historial del mejor modelo

## 📝 Nota Importante

**Este enfoque tiene mucho sentido porque:**

1. **Durante la evolución**, cada vez que un modelo supera el mejor fitness global:
   - Se guarda automáticamente su checkpoint
   - Se elimina el checkpoint anterior (ahorro de espacio)
   - Se asegura que siempre tenemos el mejor modelo disponible

2. **Para la evaluación 5-fold CV**:
   - En lugar de entrenar 5 modelos desde cero (aleatorio)
   - Se usan los pesos pre-entrenados del mejor modelo como inicio
   - Esto es **Transfer Learning**, que típicamente da mejores resultados
   - Cada fold hace fine-tuning con sus propios datos

3. **Ventajas prácticas**:
   - Si el proceso se interrumpe, no se pierde el mejor modelo
   - Se puede reanudar la evaluación 5-fold desde el checkpoint
   - Las métricas son más estables y representativas
   - Se optimiza el uso de recursos (disco y tiempo de entrenamiento)

## 10. Evaluación Completa de Métricas (Tabla)

In [ ]:
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score, 
    roc_auc_score,
    confusion_matrix,
    classification_report
)

def load_fold_data(config, fold_number):
    """
    Carga los datos de un fold específico.
    
    Args:
        config: Configuración del sistema
        fold_number: Número de fold (1-5)
    
    Returns:
        Tuple de (train_loader, test_loader) para ese fold
    """
    fold_files_directory = os.path.join(
        config['data_path'], 
        f"files_{config['fold_id']}"
    )
    
    fold_index = fold_number
    dataset_id = config['dataset_id']
    
    # Cargar datos del fold
    x_train = np.load(os.path.join(fold_files_directory, f'X_train_{dataset_id}_fold_{fold_index}.npy'))
    y_train = np.load(os.path.join(fold_files_directory, f'y_train_{dataset_id}_fold_{fold_index}.npy'))
    x_val = np.load(os.path.join(fold_files_directory, f'X_val_{dataset_id}_fold_{fold_index}.npy'))
    y_val = np.load(os.path.join(fold_files_directory, f'y_val_{dataset_id}_fold_{fold_index}.npy'))
    x_test = np.load(os.path.join(fold_files_directory, f'X_test_{dataset_id}_fold_{fold_index}.npy'))
    y_test = np.load(os.path.join(fold_files_directory, f'y_test_{dataset_id}_fold_{fold_index}.npy'))
    
    # Reshape si es necesario
    if len(x_train.shape) == 2:
        x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
        x_val = x_val.reshape((x_val.shape[0], 1, x_val.shape[1]))
        x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
    
    # Convertir a tensores
    x_train_tensor = torch.FloatTensor(x_train)
    y_train_tensor = torch.LongTensor(y_train.astype(np.int64))
    x_val_tensor = torch.FloatTensor(x_val)
    y_val_tensor = torch.LongTensor(y_val.astype(np.int64))
    x_test_tensor = torch.FloatTensor(x_test)
    y_test_tensor = torch.LongTensor(y_test.astype(np.int64))
    
    # Crear datasets
    train_dataset = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
    x_eval = torch.cat([x_val_tensor, x_test_tensor], dim=0)
    y_eval = torch.cat([y_val_tensor, y_test_tensor], dim=0)
    test_dataset = torch.utils.data.TensorDataset(x_eval, y_eval)
    
    # Crear DataLoaders
    fold_train_loader = DataLoader(
        train_dataset, 
        batch_size=config['batch_size'], 
        shuffle=True,
        num_workers=0,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    fold_test_loader = DataLoader(
        test_dataset, 
        batch_size=config['batch_size'], 
        shuffle=False,
        num_workers=0,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    return fold_train_loader, fold_test_loader


def evaluate_single_fold(best_genome, config, fold_train_loader, fold_test_loader, fold_num, num_epochs=20, use_pretrained=False, pretrained_model=None):
    """
    Entrena y evalúa el modelo en un solo fold.
    
    Args:
        best_genome: Genoma de la mejor arquitectura
        config: Configuración del sistema
        fold_train_loader: DataLoader de entrenamiento del fold
        fold_test_loader: DataLoader de test del fold
        fold_num: Número del fold
        num_epochs: Épocas de entrenamiento
        use_pretrained: Si True, usa el modelo pre-entrenado como inicio
        pretrained_model: Modelo pre-entrenado opcional
    
    Returns:
        dict: Métricas del fold
    """
    print(f"\n{'='*70}")
    print(f"FOLD {fold_num}/5")
    print(f"{'='*70}")
    
    # Crear modelo nuevo para este fold
    model = EvolvableCNN(best_genome, config).to(device)
    
    # Si hay un modelo pre-entrenado, copiar sus pesos como punto de partida
    if use_pretrained and pretrained_model is not None:
        print("   Inicializando desde modelo pre-entrenado...")
        try:
            model.load_state_dict(pretrained_model.state_dict())
            print("   ✓ Pesos pre-entrenados cargados exitosamente")
        except Exception as e:
            print(f"   ✗ Error cargando pesos pre-entrenados: {e}")
            print("   Continuando con pesos aleatorios...")
    
    # Configurar optimizer y criterion
    optimizer_class = OPTIMIZERS[best_genome['optimizer']]
    optimizer = optimizer_class(model.parameters(), lr=best_genome['learning_rate'])
    criterion = nn.CrossEntropyLoss()
    
    # Entrenamiento
    print(f"Entrenando por {num_epochs} épocas...")
    model.train()
    
    for epoch in range(1, num_epochs + 1):
        running_loss = 0.0
        batch_count = 0
        
        for data, target in fold_train_loader:
            data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            batch_count += 1
        
        avg_loss = running_loss / max(1, batch_count)
        
        if epoch % 5 == 0 or epoch == 1:
            print(f"   Época {epoch}/{num_epochs}: loss={avg_loss:.4f}")
    
    # Evaluación
    print("Evaluando...")
    model.eval()
    all_predictions = []
    all_targets = []
    all_probs = []
    
    with torch.no_grad():
        for data, target in fold_test_loader:
            data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)
            output = model(data)
            
            # Probabilidades para AUC
            probs = F.softmax(output, dim=1)
            
            # Predicciones
            _, predicted = torch.max(output, 1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    # Convertir a numpy
    y_true = np.array(all_targets)
    y_pred = np.array(all_predictions)
    y_probs = np.array(all_probs)
    
    # Calcular métricas
    accuracy = accuracy_score(y_true, y_pred) * 100
    sensitivity = recall_score(y_true, y_pred, pos_label=1, zero_division=0) * 100
    specificity = recall_score(y_true, y_pred, pos_label=0, zero_division=0) * 100
    f1 = f1_score(y_true, y_pred, zero_division=0) * 100
    
    try:
        auc = roc_auc_score(y_true, y_probs[:, 1]) * 100
    except:
        auc = 0.0
    
    cm = confusion_matrix(y_true, y_pred)
    
    print(f"\nResultados Fold {fold_num}:")
    print(f"   Accuracy:     {accuracy:.2f}%")
    print(f"   Sensitivity:  {sensitivity:.2f}%")
    print(f"   Specificity:  {specificity:.2f}%")
    print(f"   F1-Score:     {f1:.2f}%")
    print(f"   AUC:          {auc:.2f}%")
    
    return {
        'fold': fold_num,
        'accuracy': accuracy,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'f1_score': f1,
        'auc': auc,
        'confusion_matrix': cm,
        'n_samples': len(y_true)
    }


def evaluate_5fold_cross_validation(best_genome, config, num_epochs=20, neuroevolution_instance=None):
    """
    Evalúa la mejor arquitectura usando 5-fold cross-validation.
    Utiliza el checkpoint del mejor modelo si está disponible.
    
    Args:
        best_genome: Genoma de la mejor arquitectura
        config: Configuración del sistema
        num_epochs: Épocas de entrenamiento por fold
        neuroevolution_instance: Instancia de HybridNeuroevolution para cargar checkpoint
    
    Returns:
        dict: Resultados agregados de todos los folds
    """
    print("="*80)
    print("EVALUACIÓN 5-FOLD CROSS-VALIDATION")
    print("="*80)
    
    print(f"\nArquitectura a evaluar:")
    print(f"   Conv1D Layers: {best_genome['num_conv_layers']}")
    print(f"   FC Layers: {best_genome['num_fc_layers']}")
    print(f"   Optimizer: {best_genome['optimizer']}")
    print(f"   Learning Rate: {best_genome['learning_rate']}")
    print(f"   Épocas por fold: {num_epochs}")
    
    # Intentar cargar el checkpoint del mejor modelo
    pretrained_model = None
    use_pretrained = False
    
    if neuroevolution_instance is not None:
        print(f"\nIntentando cargar checkpoint del mejor modelo...")
        genome_from_checkpoint, pretrained_model = neuroevolution_instance.load_best_checkpoint()
        
        if pretrained_model is not None:
            use_pretrained = True
            print(f"✓ Checkpoint cargado exitosamente")
            print(f"  Los modelos de cada fold se inicializarán con estos pesos pre-entrenados")
        else:
            print(f"✗ No se pudo cargar checkpoint, se entrenarán desde cero")
    else:
        print(f"\nNo se proporcionó instancia de neuroevolution, entrenando desde cero")
    
    # Almacenar resultados de cada fold
    fold_results = []
    
    # Evaluar cada fold
    for fold_num in range(1, 6):  # 5 folds
        print(f"\n\nCargando datos del Fold {fold_num}...")
        
        try:
            fold_train_loader, fold_test_loader = load_fold_data(config, fold_num)
            print(f"   Train batches: {len(fold_train_loader)}")
            print(f"   Test batches: {len(fold_test_loader)}")
            
            # Evaluar este fold (usando modelo pre-entrenado si está disponible)
            fold_result = evaluate_single_fold(
                best_genome, 
                config, 
                fold_train_loader, 
                fold_test_loader, 
                fold_num, 
                num_epochs,
                use_pretrained=use_pretrained,
                pretrained_model=pretrained_model
            )
            fold_results.append(fold_result)
            
        except Exception as e:
            print(f"   ERROR en Fold {fold_num}: {e}")
            print(f"   Saltando este fold...")
            import traceback
            traceback.print_exc()
            continue
    
    # Calcular estadísticas agregadas
    print("\n\n" + "="*80)
    print("RESULTADOS AGREGADOS (5-FOLD CROSS-VALIDATION)")
    print("="*80)
    
    if not fold_results:
        print("ERROR: No se pudo evaluar ningún fold")
        return None
    
    # Extraer métricas de todos los folds
    accuracies = [r['accuracy'] for r in fold_results]
    sensitivities = [r['sensitivity'] for r in fold_results]
    specificities = [r['specificity'] for r in fold_results]
    f1_scores = [r['f1_score'] for r in fold_results]
    aucs = [r['auc'] for r in fold_results]
    
    # Calcular promedios y desviaciones estándar
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)
    
    mean_sensitivity = np.mean(sensitivities)
    std_sensitivity = np.std(sensitivities)
    
    mean_specificity = np.mean(specificities)
    std_specificity = np.std(specificities)
    
    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)
    
    mean_auc = np.mean(aucs)
    std_auc = np.std(aucs)
    
    # Mostrar resultados por fold
    print(f"\nRESULTADOS POR FOLD:")
    print(f"{'Fold':<6} {'Accuracy':<12} {'Sensitivity':<14} {'Specificity':<14} {'F1-Score':<12} {'AUC':<12}")
    print("-" * 80)
    for r in fold_results:
        print(f"{r['fold']:<6} {r['accuracy']:>6.2f}%      {r['sensitivity']:>6.2f}%        {r['specificity']:>6.2f}%        {r['f1_score']:>6.2f}%      {r['auc']:>6.2f}%")
    
    # Mostrar promedios
    print("-" * 80)
    print(f"{'Mean':<6} {mean_accuracy:>6.2f}%      {mean_sensitivity:>6.2f}%        {mean_specificity:>6.2f}%        {mean_f1:>6.2f}%      {mean_auc:>6.2f}%")
    print(f"{'Std':<6} {std_accuracy:>6.2f}%      {std_sensitivity:>6.2f}%        {std_specificity:>6.2f}%        {std_f1:>6.2f}%      {std_auc:>6.2f}%")
    
    # Resultados finales
    results = {
        'fold_results': fold_results,
        'mean_accuracy': mean_accuracy,
        'std_accuracy': std_accuracy,
        'mean_sensitivity': mean_sensitivity,
        'std_sensitivity': std_sensitivity,
        'mean_specificity': mean_specificity,
        'std_specificity': std_specificity,
        'mean_f1': mean_f1,
        'std_f1': std_f1,
        'mean_auc': mean_auc,
        'std_auc': std_auc,
        'n_folds': len(fold_results),
        'architecture': f"{best_genome['num_conv_layers']}Conv1D+{best_genome['num_fc_layers']}FC"
    }
    
    # Formato para tabla
    print("\n" + "="*80)
    print("FORMATO PARA TABLA")
    print("="*80)
    
    print(f"\nMÉTRICAS FINALES (promedio ± desviación estándar):")
    print(f"   Accuracy:     {mean_accuracy:.2f}% ± {std_accuracy:.2f}%")
    print(f"   Sensitivity:  {mean_sensitivity:.2f}% ± {std_sensitivity:.2f}%")
    print(f"   Specificity:  {mean_specificity:.2f}% ± {std_specificity:.2f}%")
    print(f"   F1-Score:     {mean_f1:.2f}% ± {std_f1:.2f}%")
    print(f"   AUC:          {mean_auc:.2f}% ± {std_auc:.2f}%")
    
    print(f"\nFORMATO PARA TABLA (valores en escala 0-1):")
    print(f"   Model: Neuroevolution-{results['architecture']}")
    print(f"   Accuracy:     {mean_accuracy/100:.2f} ({int(std_accuracy)}%)")
    print(f"   Sensitivity:  {mean_sensitivity/100:.2f} ({int(std_sensitivity)}%)")
    print(f"   Specificity:  {mean_specificity/100:.2f} ({int(std_specificity)}%)")
    print(f"   F1-Score:     {mean_f1/100:.2f} ({int(std_f1)}%)")
    print(f"   AUC:          {mean_auc/100:.2f} ({int(std_auc)}%)")
    
    print(f"\nFORMATO LaTeX:")
    latex_row = f"Neuroevolution-{results['architecture']} & {mean_accuracy/100:.2f} ({int(std_accuracy)}\\%) & {mean_sensitivity/100:.2f} ({int(std_sensitivity)}\\%) & {mean_specificity/100:.2f} ({int(std_specificity)}\\%) & {mean_f1/100:.2f} ({int(std_f1)}\\%) & {mean_auc/100:.2f} ({int(std_auc)}\\%) \\\\"
    print(f"   {latex_row}")
    
    print(f"\nFORMATO Markdown:")
    markdown_row = f"| Neuroevolution-{results['architecture']} | {mean_accuracy/100:.2f} ({int(std_accuracy)}%) | {mean_sensitivity/100:.2f} ({int(std_sensitivity)}%) | {mean_specificity/100:.2f} ({int(std_specificity)}%) | {mean_f1/100:.2f} ({int(std_f1)}%) | {mean_auc/100:.2f} ({int(std_auc)}%) |"
    print(f"   {markdown_row}")
    
    # Guardar resultados
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_file = f"5fold_cv_results_{timestamp}.json"
    
    try:
        with open(results_file, 'w') as f:
            json.dump(results, f, indent=2, default=str)
        print(f"\n✓ Resultados guardados en: {results_file}")
    except Exception as e:
        print(f"\n✗ Error guardando resultados: {e}")
    
    print("\n" + "="*80)
    
    return results


# Ejecutar evaluación 5-fold cross-validation
print("Iniciando evaluación 5-fold cross-validation de la mejor arquitectura...\n")
print("Usando el checkpoint del mejor modelo encontrado durante la evolución.\n")
cv_results = evaluate_5fold_cross_validation(best_genome, CONFIG, num_epochs=20, neuroevolution_instance=neuroevolution)

## 10. Evaluación 5-Fold Cross-Validation con Checkpoint

Esta sección evalúa la mejor arquitectura encontrada usando **5-fold cross-validation**.

### 🎯 Ventajas del enfoque con checkpoints:

1. **Eficiencia**: Se guarda el mejor modelo durante la evolución (no se reentrena desde cero)
2. **Transfer Learning**: Los pesos pre-entrenados sirven como punto de partida para cada fold
3. **Gestión de espacio**: Solo se mantiene el checkpoint del mejor modelo global
4. **Robustez**: Métricas más confiables con intervalos de confianza

### 📊 Proceso:

1. Se carga el checkpoint del mejor modelo encontrado
2. Para cada fold:
   - Se inicializa un modelo con los pesos pre-entrenados
   - Se fine-tunea con los datos de entrenamiento del fold
   - Se evalúa en los datos de test del fold
3. Se calculan métricas agregadas (promedio ± desviación estándar)